## In this notebook we'll clean and process the dataset in order to make it more suitable for the ML models

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\yagoc\OneDrive\Escritorio\pite\pite_py_projects\telco-churn-prediction\data\processed\teclo_custommer_churn_processed.csv")

### First thing, we'll change the dtype of the column TotalCharges as right now is a string

In [3]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"].astype(str).str.strip(), errors="coerce")

In [4]:
# Quick data quality overview
print("Shape:", df.shape)
print("\nDtypes:\n", df.dtypes)

# Missing values
missing = df.isna().sum().sort_values(ascending=False)
print("\nMissing values (non-zero):")
print(missing[missing > 0])

# Duplicates
print("\nDuplicate rows:", df.duplicated().sum())

# Low-cardinality object columns
obj_cols = df.select_dtypes(include=["object", "string"]).columns
low_card = {col: sorted(df[col].dropna().unique()) for col in obj_cols if df[col].nunique(dropna=True) <= 6}
print("\nLow-cardinality object columns:")
for col, vals in low_card.items():
    print(f"- {col}: {vals}")

Shape: (7043, 21)

Dtypes:
 customerID              str
gender                  str
SeniorCitizen         int64
Partner                 str
Dependents              str
tenure                int64
PhoneService            str
MultipleLines           str
InternetService         str
OnlineSecurity          str
OnlineBackup            str
DeviceProtection        str
TechSupport             str
StreamingTV             str
StreamingMovies         str
Contract                str
PaperlessBilling        str
PaymentMethod           str
MonthlyCharges      float64
TotalCharges        float64
Churn                   str
dtype: object

Missing values (non-zero):
TotalCharges    11
dtype: int64

Duplicate rows: 0

Low-cardinality object columns:
- gender: ['Female', 'Male']
- Partner: ['No', 'Yes']
- Dependents: ['No', 'Yes']
- PhoneService: ['No', 'Yes']
- MultipleLines: ['No', 'No phone service', 'Yes']
- InternetService: ['DSL', 'Fiber optic', 'No']
- OnlineSecurity: ['No', 'No internet service',

## Check for missing and duplicate values, handle them

In [5]:
missing_cols = df.isna().sum()
missing_cols = missing_cols[missing_cols > 0]

print("Columns with missing values:")
print(missing_cols)

if "TotalCharges" in missing_cols.index:
    missing_tc = df[df["TotalCharges"].isna()]
    print(missing_tc[["customerID", "tenure", "MonthlyCharges", "TotalCharges"]].head())
    print("Missing TotalCharges with tenure=0:", (missing_tc["tenure"] == 0).sum(), "of", len(missing_tc))

Columns with missing values:
TotalCharges    11
dtype: int64
      customerID  tenure  MonthlyCharges  TotalCharges
488   4472-LVYGI       0           52.55           NaN
753   3115-CZMZD       0           20.25           NaN
936   5709-LVOEQ       0           80.85           NaN
1082  4367-NUYAO       0           25.75           NaN
1340  1371-DWPAZ       0           56.05           NaN
Missing TotalCharges with tenure=0: 11 of 11


In [6]:
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [7]:
df = df.drop_duplicates()

### Checking for outliers (only numerical columns)

In [8]:
def detect_outliers_iqr(df,column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]

    print(f"{column}: {len(outliers)} outliers found")
    print(f"Lower bound: {lower_bound}, Upper bound: {upper_bound}")
    return outliers

In [9]:
numerical = ["tenure", "MonthlyCharges", "TotalCharges"]

for col in df[numerical]:
    detect_outliers_iqr(df, col)

tenure: 0 outliers found
Lower bound: -60.0, Upper bound: 124.0
MonthlyCharges: 0 outliers found
Lower bound: -46.02499999999999, Upper bound: 171.375
TotalCharges: 0 outliers found
Lower bound: -4683.525, Upper bound: 8868.675


### Next, we'll convert all the binary columns to real binary

In [10]:
mapping = {"yes": 1, "no": 0}

df = df.apply(lambda col: col.str.strip().str.lower() if col.dtype in ["object", "str"] else col)

binary_cols = [
    col for col in df.columns
    if df[col].dtype in ["object", "str"] and set(df[col].dropna().unique()) == {"yes", "no"}
]

df[binary_cols] = df[binary_cols].replace(mapping)

In [11]:
binary_cols

['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']

### Handle multi-category columns (keep yes/no/no service as separate categories)

In [ ]:
service_cols = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

# One-hot encode these columns (drop_first=True to avoid multicollinearity)
df = pd.get_dummies(df, columns=service_cols, drop_first=True, dtype=int)

new_cols = [col for col in df.columns if any(s in col for s in service_cols)]
print(new_cols)

['MultipleLines_no phone service', 'MultipleLines_yes', 'OnlineSecurity_no internet service', 'OnlineSecurity_yes', 'OnlineBackup_no internet service', 'OnlineBackup_yes', 'DeviceProtection_no internet service', 'DeviceProtection_yes', 'TechSupport_no internet service', 'TechSupport_yes', 'StreamingTV_no internet service', 'StreamingTV_yes', 'StreamingMovies_no internet service', 'StreamingMovies_yes']


### Encode remaining categorical columns

In [ ]:
df['gender'] = df['gender'].map({'female': 0, 'male': 1})

categorical_cols = ['InternetService', 'Contract', 'PaymentMethod']
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True, dtype=int)

print(f"\nAll columns:")
print(df.columns.tolist())


All columns:
['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'PaperlessBilling', 'MonthlyCharges', 'TotalCharges', 'Churn', 'MultipleLines_no phone service', 'MultipleLines_yes', 'OnlineSecurity_no internet service', 'OnlineSecurity_yes', 'OnlineBackup_no internet service', 'OnlineBackup_yes', 'DeviceProtection_no internet service', 'DeviceProtection_yes', 'TechSupport_no internet service', 'TechSupport_yes', 'StreamingTV_no internet service', 'StreamingTV_yes', 'StreamingMovies_no internet service', 'StreamingMovies_yes', 'InternetService_fiber optic', 'InternetService_no', 'Contract_one year', 'Contract_two year', 'PaymentMethod_credit card (automatic)', 'PaymentMethod_electronic check', 'PaymentMethod_mailed check']


### Feature Engineering - Create useful derived features

In [ ]:
df['AvgMonthlyCharges'] = df['TotalCharges'] / (df['tenure'].replace(0, 1))

df['TenureGroup'] = pd.cut(df['tenure'], bins=[0, 12, 24, 48, 72], 
                             labels=['0-1yr', '1-2yr', '2-4yr', '4-6yr'])

df = pd.get_dummies(df, columns=['TenureGroup'], drop_first=True, dtype=int)

# Total services count (count how many additional services customer has)
service_features = [col for col in df.columns if any(x in col for x in 
                    ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                     'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines'])]
# Only count 'yes' columns (those ending with 'yes')
yes_columns = [col for col in service_features if col.endswith('yes')]
df['TotalServices'] = df[yes_columns].sum(axis=1)

### Save the cleaned and processed dataset

In [15]:
# Drop customerID as it's not needed for modeling
df_final = df.drop('customerID', axis=1)

# Save to processed folder
output_path = r"C:\Users\yagoc\OneDrive\Escritorio\pite\pite_py_projects\telco-churn-prediction\data\processed\telco_churn_cleaned.csv"
df_final.to_csv(output_path, index=False)

print(f"✓ Data saved to: {output_path}")
print(f"\nFinal dataset summary:")
print(f"- Shape: {df_final.shape}")
print(f"- Features: {df_final.shape[1] - 1}")  # -1 for target column
print(f"- Missing values: {df_final.isna().sum().sum()}")
print(f"\nData types:")
print(df_final.dtypes.value_counts())

✓ Data saved to: C:\Users\yagoc\OneDrive\Escritorio\pite\pite_py_projects\telco-churn-prediction\data\processed\telco_churn_cleaned.csv

Final dataset summary:
- Shape: (7043, 36)
- Features: 35
- Missing values: 0

Data types:
int64      28
object      5
float64     3
Name: count, dtype: int64


In [16]:
# Display sample of the final cleaned data
print("Sample of cleaned data:")
print(df_final.head())
print("\n" + "="*80)
print("All columns in final dataset:")
print(df_final.columns.tolist())

Sample of cleaned data:
   gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0       0              0       1          0       1            0   
1       1              0       0          0      34            1   
2       1              0       0          0       2            1   
3       1              0       0          0      45            0   
4       0              0       0          0       2            1   

  PaperlessBilling  MonthlyCharges  TotalCharges Churn  ...  \
0                1           29.85         29.85     0  ...   
1                0           56.95       1889.50     0  ...   
2                1           53.85        108.15     1  ...   
3                0           42.30       1840.75     0  ...   
4                1           70.70        151.65     1  ...   

   Contract_one year  Contract_two year  \
0                  0                  0   
1                  1                  0   
2                  0                  0   
3               

---
## Summary: Data Cleaning & Feature Engineering Explained

### 🧹 Data Cleaning Steps:

**1. Type Conversion & Missing Values**
- Converted `TotalCharges` from string to numeric
- Found 11 missing values in `TotalCharges`, all with `tenure=0` (new customers)
- **Decision**: Filled with 0 (logically correct for new customers with no billing history)

**2. Duplicate Removal**
- Removed duplicate rows to ensure data quality

**3. Outlier Detection**
- Used IQR method to identify outliers in numerical columns
- Kept outliers as they represent valid extreme customer behaviors

---

### 🔢 Encoding Strategy:

**1. Binary Columns (Yes/No)**
- Columns: Partner, Dependents, PhoneService, PaperlessBilling, Churn
- Encoded as: Yes=1, No=0
- **Why**: Simple binary encoding for ML algorithms

**2. Multi-Category Service Columns (Yes/No/No Service)**
- Columns: MultipleLines, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies
- **Decision**: Kept 3 categories separate using **one-hot encoding**
- **Why**: "No internet service" vs "No" carries different information
  - "No" = Has base service, chose not to subscribe to add-on
  - "No internet/phone service" = Can't subscribe because they don't have the base service
  - This distinction may influence churn behavior differently

**3. Gender**
- Binary encoded: Female=0, Male=1

**4. Other Categorical Columns**
- InternetService (DSL, Fiber optic, No)
- Contract (Month-to-month, One year, Two year)
- PaymentMethod (4 types)
- **Method**: One-hot encoding with `drop_first=True`
- **Why**: Avoids multicollinearity (dummy variable trap) while preserving all information

---

### 🛠️ Feature Engineering:

**1. AvgMonthlyCharges**
- Formula: `TotalCharges / tenure` (with safeguard for tenure=0)
- **Purpose**: Captures average spending behavior, useful for identifying high-value customers

**2. TenureGroup**
- Bins: 0-1yr, 1-2yr, 2-4yr, 4-6yr
- **Purpose**: Capture non-linear relationships between tenure and churn
  - New customers (0-1yr) may have different churn patterns than long-term customers

**3. TotalServices**
- Count of additional services customer has subscribed to
- **Purpose**: Customer engagement indicator
  - More services → more "locked in" → potentially lower churn
  - Hypothesis: Customers with more services have lower churn rates

---

### 📊 Final Dataset:
- All features are now numerical (ready for ML models)
- No missing values
- CustomerID dropped (not predictive)